# Playground SQL

Pour rappel, on a 4 tables suivantes:

- Athlete : id (primary key), name, gender, team
- Event : id (primary key), location, year, distance, stroke, relay
- Team : id (primary key), event_id (foreign key), athlete_id (foreign key)
- Results : id (primary key), team_id (foreign key), results, rank, quit_reason

Le but ici va être de s'entraîner sur des requêtes SQL avancées.

In [ ]:
# Connexion à la base de données
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

# Charger les variables d'environnement
load_dotenv()

username = os.getenv("DB_USERNAME")
password = os.getenv("DB_PASSWORD")
host = os.getenv("DB_HOST")
port = os.getenv("DB_PORT")
dbname = os.getenv("DB_NAME")

# Création de la base de données
engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{dbname}')


GPT o1-mini, avec la connaissance du contexte, nous a concocté quelques exercices :

Voici une série de défis SQL conçus pour vous aider à vous entraîner et à maîtriser l'utilisation des Common Table Expressions (CTEs), des fonctions de fenêtre (window functions) et des classements (RANKs) dans le contexte de votre base de données sur les résultats de natation olympique.

1. Top 3 Performers par Année et Style de Nage

    Écrivez une requête SQL utilisant un CTE et des fonctions de fenêtre pour identifier les trois athlètes les plus performants (les résultats les plus bas) dans chaque style de nage pour chaque année olympique.

2. Moyenne des Résultats par Équipe

    Utilisez une fonction de fenêtre pour calculer le temps moyen des résultats pour chaque équipe à travers tous les événements et listez les équipes dont la performance est supérieure à la moyenne générale.

3. Amélioration des Performances au Fil du Temps

    Employez un CTE pour organiser les données par athlète et année, puis utilisez des fonctions de fenêtre pour calculer la différence de résultats par rapport à l'année olympique précédente pour chaque athlète.

4. Classement de Participation aux Événements avec SELF JOIN

    **Énoncé :**

    Classez les athlètes en fonction du nombre d'événements auxquels ils ont participé en utilisant `RANK` ou `DENSE_RANK`. Utilisez un `SELF JOIN` pour comparer les participations entre les athlètes et listez les dix athlètes les plus actifs.

    **Objectifs :**
    - Utiliser `RANK` ou `DENSE_RANK`
    - Utiliser des CTEs et des fonctions de fenêtre

5. Analyse des Équipes de Relais avec CROSS JOIN

    **Énoncé :**

    Utilisez un `CROSS JOIN` pour combiner toutes les équipes de relais avec les événements et déterminez quelles équipes ont les résultats combinés les plus rapides. Classez-les en conséquence en utilisant `RANK`.

    **Objectifs :**
    - Utiliser `CROSS JOIN`
    - Appliquer `RANK`
    - Utiliser des CTEs et des fonctions de fenêtre

6. Domination des Équipes par Style de Nage avec RANK

    **Énoncé :**

    Identifiez quelles équipes ont remporté le plus de médailles dans chaque catégorie de style de nage à travers toutes les années en utilisant `RANK`. Utilisez des CTEs pour préparer les données nécessaires.

    **Objectifs :**
    - Utiliser `RANK`
    - Implémenter des CTEs
    - Utiliser des fonctions de fenêtre

7. Comparaison des Performances avec DENSE_RANK et SELF JOIN

    **Énoncé :**

    Comparez les performances des athlètes en utilisant `DENSE_RANK` pour classer les résultats et un `SELF JOIN` pour comparer les performances entre athlètes similaires. Identifiez les athlètes dont les performances sont consistantes ou en amélioration.

    **Objectifs :**
    - Utiliser `DENSE_RANK`
    - Implémenter un `SELF JOIN`
    - Utiliser des CTEs et des fonctions de fenêtre

8. Amélioration des Performances par Genre avec CROSS JOIN

    **Énoncé :**

    Utilisez un `CROSS JOIN` pour combiner les données des athlètes masculins et féminins. Comparez les résultats moyens entre les genres pour chaque année en utilisant des fonctions de fenêtre et classez l'évolution des performances par genre avec `RANK`.

    **Objectifs :**
    - Utiliser `CROSS JOIN`
    - Appliquer `RANK`
    - Utiliser des CTEs et des fonctions de fenêtre

9. Plus Grande Amélioration des Résultats avec DENSE_RANK et CTE

    **Énoncé :**

    Identifiez les athlètes qui ont montré la plus grande amélioration dans leurs résultats au fil des Jeux Olympiques successifs en utilisant `DENSE_RANK`. Utilisez des CTEs pour organiser les données par athlète et année, puis appliquez des fonctions de fenêtre pour calculer les améliorations.

    **Objectifs :**
    - Utiliser `DENSE_RANK`
    - Implémenter des CTEs
    - Utiliser des fonctions de fenêtre

10. Consistance des Performances avec RANK et SELF JOIN

    **Énoncé :**

    Mesurez la consistance de chaque athlète en calculant l'écart type de leurs temps de résultats à travers différents événements. Utilisez `RANK` pour classer les athlètes du plus au moins constant en utilisant un `SELF JOIN` pour comparer les écarts types.

    **Objectifs :**
    - Utiliser `RANK`
    - Implémenter un `SELF JOIN`
    - Utiliser des CTEs et des fonctions de fenêtre

---
Ces défis vous permettront de manipuler et d'analyser vos données de manière approfondie tout en renforçant vos compétences en SQL avancé. Bon entraînement !

## Exercice 1 : Top 3 Performers par Année et Style de Nage

"Écrivez une requête SQL utilisant un CTE et des fonctions de fenêtre pour identifier les trois athlètes les plus performants (les résultats les plus bas) dans chaque style de nage pour chaque année olympique."

On va pas compter les relais en revanche, parce qu'on a pas l'info de qui dans l'équipe a été meilleur dans le dataset.

On va aussi prendre en compte la distance, ça n'a pas de sens de comparer les résultats UNIQUEMENT par style de nage.

Voici le résultat en prenant en compte toutes les données, pas besoin de CTE ou de window function c'est trivial.

In [ ]:
from sqlalchemy import text

session = engine.connect()

query = text("""
SELECT a.name, e.stroke, e.distance, e.year, r.rank
FROM athletes a
INNER JOIN event_teams t ON a.athlete_id = t.athlete_id
INNER JOIN events e ON t.event_id = e.event_id
INNER JOIN results r ON t.event_team_id = r.event_team_id
WHERE NOT e.is_relay
AND r.rank <= 3
AND r.rank > 0
ORDER BY e.year, e.stroke, e.distance, r.rank
""")

result = session.execute(query)
print(result.fetchall())

session.close()

On va maintenant oublier que le rank a déjà été calculé et calculer cela nous-mêmes, on prendra quand même soin d'éliminer les tuples qui ont été disqualifiés pour prévenir tout résultat biaisé.

In [ ]:
from sqlalchemy import text

session = engine.connect()

query = text("""
WITH QualifiedSoloAthletes AS (
    SELECT
        a.athlete_id, e.stroke, e.distance, e.year, a.name, r.results,
        RANK() OVER (PARTITION BY year, stroke, distance ORDER BY results ASC) AS rank
    FROM athletes a
    INNER JOIN event_teams t ON a.athlete_id = t.athlete_id
    INNER JOIN events e ON t.event_id = e.event_id
    INNER JOIN results r ON t.event_team_id = r.event_team_id
    WHERE NOT e.is_relay
    AND r.rank > 0
)
SELECT 
    name, stroke, distance, year, rank
FROM QualifiedSoloAthletes
WHERE rank <= 3
ORDER BY year, stroke, distance, rank
""")

result = session.execute(query)
print(result.fetchall())

session.close()

## Exercice 2 : Moyenne des Résultats par Équipe

Utilisez une fonction de fenêtre pour calculer le temps moyen des résultats pour chaque équipe à travers tous les événements et listez les équipes dont la performance est supérieure à la moyenne générale.

Pour faire cette comparaison, il faut comparer type d'épreuve par type d'épreuve, et donc pour chaque type d'épreuve, calculer la moyenne des résultats.

On va donc GROUP BY épreuve.

Petite erreur de conception : pour chaque athlète y a une colonne "team", alors que d'une part un athlète peut avoir différentes équipes au cours du temps (un binational par exemple), et d'autre part tous les membres d'une "team" au sens actuel, donc les coéquipiers sur une épreuve, sont forcément de la même équipe nationale.

C'est corrigé, maintenant y a une table event_teams, et une table national_teams.

Pour l'exercice 2, on va donc calculer la moyenne des résultats pour chaque équipe sur chaque type d'épreuve, et on va filtrer les équipes dont la performance est supérieure à la moyenne générale.


In [ ]:
from sqlalchemy import text

session = engine.connect()

query = text("""
WITH AverageResultsPerEvent AS (
    SELECT
        e.stroke,
        e.distance,
        e.nb_relay,
        AVG(r.results) AS average_results
    FROM (SELECT * FROM results WHERE nullif(results, 'NaN') is not null and rank > 0) r
    INNER JOIN event_teams et ON r.event_team_id = et.event_team_id
    INNER JOIN events e ON et.event_id = e.event_id
    GROUP BY e.stroke, e.distance, e.nb_relay
),
AverageResultsPerEventAndNationalTeam AS (
    SELECT
        nt.code,
        e.stroke,
        e.distance,
        e.nb_relay,
        AVG(r.results) AS average_results
    FROM (SELECT * FROM results WHERE nullif(results, 'NaN') is not null and rank > 0) r
    INNER JOIN event_teams et ON r.event_team_id = et.event_team_id
    INNER JOIN events e ON et.event_id = e.event_id
    INNER JOIN national_teams nt ON et.national_team_id = nt.national_team_id
    GROUP BY e.stroke, e.distance, e.nb_relay, nt.code
)

SELECT a.stroke, a.nb_relay, a.distance, a.code, a.average_results, b.average_results
FROM AverageResultsPerEventAndNationalTeam a
INNER JOIN AverageResultsPerEvent b ON
    a.stroke = b.stroke
    AND a.distance = b.distance
    AND a.nb_relay = b.nb_relay
WHERE a.average_results < b.average_results
ORDER BY a.stroke, a.nb_relay, a.distance, a.average_results
""")

result = session.execute(query)
print(result.fetchall())

session.close()

## Exercice 3 : Amélioration des Performances au Fil du Temps

Employez un CTE pour organiser les données par athlète et année, puis utilisez des fonctions de fenêtre pour calculer la différence de résultats par rapport à l'année olympique précédente pour chaque athlète.

Je vais préciser l'énoncé : on prend la dernière année olympique qu'a couru un athlète sur une épreuve donnée, et on compare avec la précédente année olympique qu'il a courue s'il y en a une. Sinon, on ne le prend pas en compte.

In [ ]:
from sqlalchemy import text

session = engine.connect()

query = text("""
WITH LastOlympicYear AS (
    SELECT
        et.athlete_id,
        MAX(e.year) AS last_year
    FROM events e
    INNER JOIN event_teams et ON e.event_id = et.event_id
    WHERE e.stroke = :stroke
    AND e.distance = :distance
    AND e.nb_relay = :nb_relay
    GROUP BY et.athlete_id
),
PreviousOlympicYear AS (
    SELECT
        et.athlete_id,
        MAX(e.year) AS previous_year
    FROM events e
    INNER JOIN event_teams et ON e.event_id = et.event_id
    INNER JOIN LastOlympicYear l ON et.athlete_id = l.athlete_id
    WHERE e.year < l.last_year
    AND e.stroke = :stroke
    AND e.distance = :distance
    AND e.nb_relay = :nb_relay
    GROUP BY et.athlete_id
),
ResultsPerAthletePerYear AS (
    SELECT
        et.athlete_id,
        e.year,
        r.results
    FROM events e
    INNER JOIN event_teams et ON e.event_id = et.event_id
    INNER JOIN results r ON et.event_team_id = r.event_team_id
    WHERE e.stroke = :stroke
    AND e.distance = :distance
    AND e.nb_relay = :nb_relay
    AND r.rank > 0
)

SELECT
    a.name,
    l.last_year,
    r_last.results AS last_results,
    p.previous_year,
    r_prev.results AS previous_results,
    (r_last.results - r_prev.results) AS results_difference
FROM LastOlympicYear l
LEFT JOIN ResultsPerAthletePerYear r_last ON l.athlete_id = r_last.athlete_id AND l.last_year = r_last.year
LEFT JOIN PreviousOlympicYear p ON l.athlete_id = p.athlete_id
LEFT JOIN ResultsPerAthletePerYear r_prev ON p.athlete_id = r_prev.athlete_id AND r_prev.year = p.previous_year
INNER JOIN athletes a ON p.athlete_id = a.athlete_id
WHERE r_last.results IS NOT NULL AND r_prev.results IS NOT NULL
ORDER BY a.name
""")

nb_relay = 1
distance = 100
stroke = "Breaststroke"

result = session.execute(query, {"stroke": stroke, "distance": distance, "nb_relay": nb_relay})
results = result.fetchall()

for row in results:
    print(row[0])

session.close()

## Exercice 4 : Classement de Participation aux Événements avec SELF JOIN

**Énoncé :**

Classez les athlètes en fonction du nombre d'événements auxquels ils ont participé en utilisant `RANK` ou `DENSE_RANK`. Utilisez un `SELF JOIN` pour comparer les participations entre les athlètes et listez les dix athlètes les plus actifs.

**Objectifs :**
- Utiliser `RANK` ou `DENSE_RANK`
- Utiliser des CTEs et des fonctions de fenêtre

In [ ]:
from sqlalchemy import text

session = engine.connect()

query = text("""
SELECT
    a.name,
    et.athlete_id,
    COUNT(et.event_id) AS number_of_events
FROM event_teams et
INNER JOIN athletes a ON et.athlete_id = a.athlete_id
GROUP BY a.name,et.athlete_id
ORDER BY number_of_events DESC
LIMIT 10
""")

result = session.execute(query)
results = result.fetchall()

print(results)

session.close()

Comme vous pouvez le constater, et à première vue, il est inutile d'utiliser des CTEs ou encore RANK() ou DENSE_RANK() pour faire ça.

Maintenant on se rend compte que c'est pas si simple que ça. Il y a beaucoup d'ex-aequo, et ne pas les prendre en compte ce serait potentiellement privilégier un athlète parce qu'il a un prénom plus proche de A qu'un autre.

On refait donc la requête en utilisant RANK() pour qu'on ait vraiment les 10 premiers, avec la possibilité d'en avoir un peu plus si il y a beaucoup d'ex-aequo. Avec DENSE_RANK() on aurait potentiellement plus d'athlètes dans le classement, car quand il y a des ex-aequo, il n'y a pas de saut de rang.

Quant au SELF JOIN c'était une hallucination de GPT, il n'y a aucun intérêt à l'utiliser ici.

In [ ]:
from sqlalchemy import text
session = engine.connect()
query = text("""
WITH athlete_event_counts AS (
    SELECT
        et.athlete_id,
        COUNT(et.event_id) AS number_of_events
    FROM event_teams et
    GROUP BY et.athlete_id
),
ranked_athletes AS (
    SELECT
        a1.athlete_id,
        a1.number_of_events,
        RANK() OVER (ORDER BY a1.number_of_events DESC) AS rank
    FROM athlete_event_counts a1
)
SELECT
    a.name,
    r.number_of_events,
    r.rank
FROM ranked_athletes r
INNER JOIN athletes a ON a.athlete_id = r.athlete_id
WHERE r.rank <= 10
ORDER BY number_of_events DESC
""")

result = session.execute(query)
results = result.fetchall()

for row in results:
    print(f"[{row[2]}] {row[0]} a participé à {row[1]} compétitions")
    
session.close()

## Exercice 5 : Analyse des Équipes de Relais avec CROSS JOIN

**Énoncé :**

Utilisez un `CROSS JOIN` pour combiner toutes les équipes de relais avec les événements et déterminez quelles équipes ont les résultats combinés les plus rapides. Classez-les en conséquence en utilisant `RANK`.

**Objectifs :**
- Utiliser `CROSS JOIN`
- Appliquer `RANK`
- Utiliser des CTEs et des fonctions de fenêtre

In [ ]:
from sqlalchemy import text

session = engine.connect()

query = text("""

""")

result = session.execute(query, {"nb_relay": 1, "stroke": "Breaststroke", "distance": 100})
results = result.fetchall()

print(results)

session.close()

# Exercice 10 : Consistance des Performances avec RANK et SELF JOIN

**Énoncé :**

Mesurez la consistance de chaque athlète en calculant l'écart type de leurs temps de résultats à travers différents événements. Utilisez `RANK` pour classer les athlètes du plus au moins constant ~~en utilisant un `SELF JOIN` pour comparer les écarts types~~.

**Objectifs :**
- Utiliser `RANK`
- ~~Implémenter un `SELF JOIN`~~
- Utiliser des CTEs et des fonctions de fenêtre

GPT a encore halluciné concernant le SELF JOIN

In [ ]:
from sqlalchemy import text

session = engine.connect()

query = text("""
WITH AthleteStdDev AS (
    SELECT
        et.athlete_id,
        STDDEV(r.results) AS ecart_type
    FROM event_teams et
    INNER JOIN events e ON et.event_id = e.event_id
    INNER JOIN results r ON et.event_team_id = r.event_team_id
    WHERE e.nb_relay = :nb_relay
    AND e.stroke = :stroke
    AND e.distance = :distance
    GROUP BY et.athlete_id
)

SELECT
    a.name,
    std.ecart_type,
    RANK() OVER (ORDER BY std.ecart_type ASC) AS rank
FROM AthleteStdDev std
INNER JOIN athletes a ON std.athlete_id = a.athlete_id
WHERE std.ecart_type IS NOT NULL
ORDER BY std.ecart_type ASC
""")

result = session.execute(query, {"nb_relay": 1, "stroke": "Breaststroke", "distance": 100})
results = result.fetchall()

print(results)

session.close()